# **Exercício pivoteamento PLU 2D - exercício slide**

Problema de LaPlace:

\begin{equation*}
  \begin{cases}
    \displaystyle \nabla^2U = 0 \text{ para } \Omega \\\\
    \displaystyle U = 0 \text{ para } \Omega_{A,B,C}\\\\
    \displaystyle U = sen(2x) \text{ para } \Omega_D
  \end{cases}
\end{equation*}

Dado que podemos reescrever a estrutura do problema como:

$AU=F$

Para o montagem do sistema de matrizes para o pivoteamento PLU, temos que a matriz A tem tamanho $(M-2)^2(M-2)^2$ e segue o seguinte padrão:

\begin{equation*}
  A = \frac{1}{h^2}
  \begin{bmatrix}
    B & -I &\\
    -I & B & -I & \\
    &  ... & ... & ... &  \\
    &   &  & -I & B \\
  \end{bmatrix}_{(M-2)^2X(M-2)^2}
\end{equation*}

Já a matriz F, segue o modelo:

\begin{equation*}
  f =
  \begin{bmatrix}
   f_1 + (\alpha_0 + \alpha_1)/h^2  \\
   f_2 + \alpha_2/h^2 \\
   ...  \\
   f_{M-2} + \alpha_{M-2}/h^2     \\
   f_{M-1} + (\alpha_{M-1} + \alpha_M)/h^2 \\
  \end{bmatrix}
\end{equation*}

In [9]:
%%writefile lu.h
#ifndef LU_H
#define LU_H

int detInd(int i, int j, int n);
void montaA(float *A, int n, float h, float *wb, float cb, float k, int *dom);
void montaF(float *F, int n, float h, int ua, int ub, float *qm, float *wb, float cb, float Ta, float k, int *dom);
void salvaMatriz1D(float *matriz, int n, const string& arquivoName);

#endif //LU_H

Overwriting lu.h


In [10]:
%%writefile lu.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>
#include <string>

#include "pivoteamento.h"

void salvaMatriz1D(float *mat, int n, string arquivoName) {
    // Matriz que vai receber os valores dos tecidos
    vector<float> matriz;

    // Inicializar a matriz que vai ser salva com elementos
    matriz.resize(n);
    for (int i = 0; i < n; i++) {
      matriz[i] =  mat[i];
    }

    // Salvar a matriz de tecidos em um arquivo para ser importada posteriormente
    ofstream arquivo(arquivoName);
    if (arquivo.is_open()) {
      for (const auto& linha : matriz) {
        arquivo << linha << " ";
      }
      arquivo.close();
      cout << endl;
      cout << "C++: Matriz solucao salva em " << arquivoName << endl;
    } else {
      cerr << "C++: Erro ao abrir o arquivo" << endl;
    }
}

int detInd(int i, int j, int n) {
  return (i < 100 && j < 100) ? i*n + j : -1;
}

void montaA(float *A, int n, float h, float *wb, float cb, float k, int *dom) {

  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {

      if(j>=i-1 && j<i+2) {
          int tecido = dom[i]; //busca o tecido em questao para definir as variaveis

          A[detInd(i,j,n)] = i==j ?  -wb[tecido]*cb*1000*h*h -2*k : k;
      }
      else A[detInd(i,j,n)] = 0;
    }
  }

  cout << "Matriz A montada: " << endl;
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cout << A[detInd(i,j,n)] << " ";
    }
    cout << endl;
  }

  salvaMatriz2D(A,n,"matA.txt");
}

void montaF(float *F, int n, float h, int ua, int ub, float *qm, float *wb, float cb, float Ta, float k, int *dom) {

  for(int i=1; i<n; i++) {
    int tecido = dom[i]; //busca o tecido em questao para definir as variaveis

    F[i] = (h*h)*(-qm[tecido] -wb[tecido]*1000*cb*Ta);
  }

  F[0] = (h*h)*(-qm[dom[0]] -wb[dom[0]]*cb*1000*Ta - (k/(h*h))*ua);
  F[n-1] = (h*h)*(-qm[dom[n-1]] -wb[dom[n-1]]*cb*1000*Ta - (k/(h*h))*ub);

  cout << endl;
  cout << "Matriz F montada: " << endl;
  for(int i=0; i<n; i++) {
    cout << F[i] << " ";
  }

  salvaMatriz1D(F,n,"montaF.txt");
}

Overwriting lu.cpp


In [11]:
%%writefile pivoteamento.h
#ifndef PIVOTEAMENTO_H
#define PIVOTEAMENTO_H

void substituicaoSucess(float *A, float *F, int *p, int n, float *x);
void substituicaoRetroa(float *A, float *F, float *x, int *p, int n);
void montaPLU(float *A, float *F, float *u, int n, int *p);
void salvaMatriz2D(float *matriz, int n,  const string& arquivo);
void salvaMatriz(float *matriz, int n, const string& arquivoName);

#endif //PIVOTEAMENTO_H

Overwriting pivoteamento.h


In [12]:
%%writefile pivoteamento.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>
#include <math.h>

#include "lu.h"

//SUBSTITUICAO SUCESSIVA Lx = pF
void substituicaoSucess(float *A, float *F, int *p, int n, float *x) {
  for(int k=0; k<n; k++) {
    x[k] = F[p[k]];

    for(int j=0; j<k; j++) {
      x[k] = x[k] - A[detInd(p[k],j,n)]*x[j];
    }
    x[k] = x[k]/1.0; //pode retirar
  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz y: " << endl;
  for(int i=0; i<n; i++) {
    cout << x[i] << " ";
  }
}

//SUBSTITUICAO RETROATIVA Ux = F
void substituicaoRetroa(float *A, float *F, float *x, int *p, int n) {
  for(int k=n-1; k>=0; k--) {

    for(int j=k+1; j<n; j++) {
      x[k] = x[k] -A[detInd(p[k],j,n)]*x[j];
    }
    x[k] = x[k]/A[detInd(p[k],k,n)];

  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz solucao: " << endl;
  for(int i=0; i<n; i++) {
    cout << x[i] << " ";
  }
}


void montaPLU(float *A, float *F, float *u, int n, int *p) {
  for(int k=0; k<n; k++) {
    //ENCONTRAR PIVOS DE A
    float max = fabs(A[detInd(p[k],k,n)]);
    int iMax = k;

    for(int i=k+1; i<n; i++) {
      if(max<fabs(A[detInd(p[i],k,n)])) {
        max = fabs(A[detInd(p[i],k,n)]);
        iMax = i;
      }
      //cout << endl;
      //cout << endl;
      //cout << endl;
      //cout << "Maior pivo: " << max << endl;
    }
    //cout << "Indice: " << iMax << endl;

    int tmp = p[k];
    p[k] = p[iMax];
    p[iMax] = tmp;

    //anular elementos
    for(int i=k+1; i<n; i++) {
      float m = A[detInd(p[i],k,n)]/A[detInd(p[k],k,n)];
      //F[i] = F[p[k]] - F[p[i]];
      A[detInd(p[i],k,n)] = m;
      for(int j=k+1; j<n; j++) {
        A[detInd(p[i],j,n)] = A[detInd(p[i],j,n)] - (m*A[detInd(p[k],j,n)]);
      }

    }
  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz A(LU): " << endl;
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cout << A[detInd(p[i],j,n)] << ", ";
    }
    cout << endl;
  }
}

Overwriting pivoteamento.cpp


In [13]:
%%writefile main.cpp
#include <iostream>
using namespace std;
#include <string>
#include <cmath>

#include "lu.h"
#include "pivoteamento.h"

int main()
{
    //PROPRIEDADES DA EQUACAO
    int n = 5; //tamanho do dominio
    float h = 0.25;
    int ua = 0; //condicao de contorno de dirichelet a esquerda
    int ub = 0; //condicao de contorno de dirichelet a topo
    int uc 0; //condicao de contorno de dirichelet a direita
    int ud = sin(2*M_PI);//condicao de contorno de dirichelet a bottom

    float *A = new float[pow(n,2)*pow(n,2)]; //matriz A
    float *F = new float[pow(n,2)]; //matriz F
    float *u = new float[n]; //vetor solucao
    int *p = new int[(n)]; //vetor para pivoteamento

    for(int i=0; i<n; i++) {
      p[i] = i;
    }

    float *x = new float[n+2];

    for(int i=0; i<n; i++) {
      x[i] = 0;
    }

    montaA(A, n, h, wb, cb, k, dom); //monta a matriz A
    montaF(F, n, h, ua, ub, qm, wb, cb, Ta, k, dom); //monta a matriz F
    montaPLU(A, F, u, n, p); //decomposicao LU
    substituicaoSucess(A, F, p, n, x); //substituicao
    substituicaoRetroa(A, F, x, p, n); //retrosubstituicao com resultado


    //aplica as condicoes de contorno
    //u[0] = ua;
    //u[n+1] = ub;

    delete [] A;
    delete [] F;
    delete [] u;
    delete [] p;

    return 0;
}

Overwriting main.cpp
